In [365]:
print("ok")

ok


In [366]:
%pwd

'c:\\'

In [355]:
import os
os.chdir("../")

In [357]:
%pwd

'c:\\'

In [358]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [359]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [367]:
os.chdir('C:/Users/Daniel/Documents/Repos/medicalchatbot')


In [368]:
extracted_data=load_pdf_file(data='Data/')

DEBUG:langchain_community.document_loaders.directory:Processing file: Data\Medical_book.pdf
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete


In [16]:
#extracted_data

In [369]:
def text_split (extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks


In [370]:
text_chunks=text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 6970


In [371]:
#text_chunks


In [372]:
from langchain.embeddings import HuggingFaceEmbeddings

In [373]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [374]:
embeddings=download_hugging_face_embeddings()

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [375]:
query_result =embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [376]:
#query_result

In [377]:
from dotenv import load_dotenv
load_dotenv()

True

In [378]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')



In [379]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc=Pinecone(api_key=PINECONE_API_KEY)

index_name="medicalchatbot2"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)




INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['c:\\Users\\Daniel\\anaconda3\\envs\\medicalbot\\lib\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into PineconeGRPC
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"name":"medicalchatbot2","metric":"cosine","dimension":384,"status":{"ready":false,"state":"Initializing"},"host":"medicalchatbot2-7hvbgxn.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"name":"medicalchatbot2","metric":"cosine","dimension":384,"status":{"ready":false,"state":"Initializing"},"host":"medicalchatbot2-7hvbgxn.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'
DEBUG:pin

In [380]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [381]:
from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)


INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['c:\\Users\\Daniel\\anaconda3\\envs\\medicalbot\\lib\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"indexes":[{"name":"medicalchatbot2","metric":"cosine","dimension":384,"status":{"ready":true,"state":"Ready"},"host":"medicalchatbot2-7hvbgxn.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}]}'
INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['c:\\Users\\Daniel\\anaconda3\\envs\\medicalbot\\lib\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"upsertedCou

In [382]:
from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['c:\\Users\\Daniel\\anaconda3\\envs\\medicalbot\\lib\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"indexes":[{"name":"medicalchatbot2","metric":"cosine","dimension":384,"status":{"ready":true,"state":"Ready"},"host":"medicalchatbot2-7hvbgxn.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}]}'
INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['c:\\Users\\Daniel\\anaconda3\\envs\\medicalbot\\lib\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference


In [384]:
docsearch

In [385]:
retriever =docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [386]:
retrieved_docs =retriever.invoke("what is Acne?")


DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"results":[],"matches":[{"id":"519e4719-d199-4c92-9298-a326223f500f","score":0.445571691,"values":[],"metadata":{"page":425,"source":"Data\\\\Medical_book.pdf","text":"Corticosteriod\xe2\x80\x94A group of synthetic hormones\\nthat are used to prevent or reduce inflammation.\\nToxic effects may result from rapid withdrawal after\\nprolonged use or from continued use of large doses.\\nPatch test\xe2\x80\x94A skin test that is done to identify aller-\\ngens. A suspected substance is applied to the skin.\\nAfter 24\xe2\x80\x9348 hours, if the area is red and swollen,\\nthe test is positive for that substance. If no reaction\\noccurs, another substance is applied. This is con-"}},{"id":"aa8afec4-4a81-4162-a39a-25a1b8076326","score":0.441253841,"values":[],"metadata":{"page":298,"source":"Data\\\\Medical_book.pdf","text":"Corticosteroids\xe2\x80\x94A group of anti-inflammatory\\nsubstances often used to treat skin conditions.\\nImmune

In [387]:
retrieved_docs

[Document(metadata={'page': 425.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Corticosteriod—A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(metadata={'page': 298.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Corticosteroids—A group of anti-inflammatory\nsubstances often used to treat skin conditions.\nImmune response—The protective reaction by the\nimmune system against foreign antigens (sub-\nstances that the body perceives as potentially dan-\ngerous). The immune system combats disease by\nneutralizing or destroying antigens.\ncontact dermatitis becomes

In [389]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is cancer",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

DEBUG:groq._base_client:Request options: {'method': 'post', 'url': '/openai/v1/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is cancer'}], 'model': 'llama3-8b-8192'}}
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:connect_tcp.started host='api.groq.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:close.complete
DEBUG:groq._base_client:Sending HTTP Request: POST https://api.groq.com/openai/v1/chat/completions
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000016E64F5A0B0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000016E3A108840> server_hostname='api.groq.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000016E39FBD870>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBU

Cancer is a group of diseases characterized by the uncontrolled growth and spread of abnormal cells. If left untreated, these abnormal cells can invade and damage healthy tissues and organs, leading to a range of health problems and even death.

Cancer occurs when there is a mutation or change in the DNA of a cell, causing it to grow and multiply uncontrollably. This process is usually triggered by a combination of genetic and environmental factors, such as:

1. Genetic mutations: Errors in the genetic code can occur spontaneously or be inherited from parents.
2. Genetic predisposition: Certain genes can increase the risk of developing cancer.
3. Environmental exposures: Exposure to carcinogens, such as tobacco smoke, radiation, and certain chemicals, can increase the risk of cancer.
4. Lifestyle factors: Poor diet, lack of physical activity, and obesity can increase the risk of cancer.

When cancer cells grow and multiply, they can form a tumor, which can:

1. Remain localized: The tu

In [390]:
from langchain.llms.base import LLM
from pydantic import BaseModel
from groq import Groq
import os

class GroqLLM(LLM, BaseModel):  # Inherit from both LLM and BaseModel
    model: str = "llama3-8b-8192"  # Define as a pydantic field
    
    # Use __annotations__ to indicate that 'client' is not a pydantic field
    client: Groq = None  # This ensures that 'client' is not managed by pydantic

    def __init__(self, **kwargs):
        super().__init__(**kwargs)  # Ensure BaseModel initialization
        # Initialize the Groq client inside __init__
        self.client = Groq(
            api_key=os.getenv("GROQ_API_KEY"),
        )

    def _call(self, prompt: str, stop=None):
        response = self.client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=self.model,
        )
        return response.choices[0].message.content

    @property
    def _llm_type(self):
        return "Groq"

# Initialize the custom LLM
llm = GroqLLM()

# Proceed with setting up the retrieval and other LangChain components as needed


In [395]:
from langchain.chains import RetrievalQA
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt =(
     "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [396]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)


In [397]:
response=rag_chain.invoke({"input":" What are Calcium channel blocker?"})

print(response["answer"])

DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"results":[],"matches":[{"id":"88c0ea11-df04-49ea-938b-c7f893b19a20","score":0.788967073,"values":[],"metadata":{"page":15,"source":"Data\\\\Medical_book.pdf","text":"CAH see Congenital adrenal hyperplasia\\nCaisson disease see Decompression\\nsickness\\nCalcaneal spurs see Heel spurs\\nCalcitonin see Bone disorder drugs\\nCalcium carbonate see Antacids\\nCalcium channel blockers\\nDefinition\\nCalcium channel blockers are medicines that slow\\nthe movement of calcium into the cells of the heart and\\nblood vessels. This, in turn, relaxes blood vessels,\\nincreases the supply of oxygen-rich blood to the heart,\\nand reduces the heart\xe2\x80\x99s workload.\\nPurpose"}},{"id":"b926b2b4-2547-43dd-b8b7-916422c01281","score":0.752161801,"values":[],"metadata":{"page":15,"source":"Data\\\\Medical_book.pdf","text":"Purpose\\nCalcium channel blockers are used to treat high\\nblood pressure, to correct abnormal heart rhythms, and to\\nr

Calcium channel blockers are medicines that slow the movement of calcium into the cells of the heart and blood vessels, which relaxes blood vessels, increases oxygen-rich blood supply to the heart, and reduces the heart's workload. They are used to treat high blood pressure, abnormal heart rhythms, angina pectoris, panic attacks, bipolar disorder, and prevent migraine headaches.
